In [217]:
import time
import re
from bs4 import BeautifulSoup
from selenium import webdriver

In [218]:
def login(username, password, url, headless=False):
    time_start = time.time()
    # Headless mode. True = hide browser
    if headless:
        option = webdriver.ChromeOptions()
        option.add_argument('headless')
        driver = webdriver.Chrome(chrome_options=option)
    else:
        driver = webdriver.Chrome()
    
    driver.get(url)
    
    # Selenium find input elements by Xpath
    time.sleep(1)
    driver.find_element_by_xpath("//div[@tyc-event-ch='Login.PasswordLogin']").click()
    driver.find_elements_by_xpath("//input[@placeholder='请输入11位手机号码']")[-2].send_keys(username)
    driver.find_element_by_xpath("//input[@placeholder='请输入登录密码']").send_keys(password)
    
    driver.execute_script("window.scrollTo(0, 60);")
    driver.find_element_by_xpath("//div[@tyc-event-ch='Login.PasswordLogin.Login']").click()
    
    time_end = time.time()
    print('Login time: {} second(s)'.format(int(time_end - time_start)))
    return driver  
                

In [219]:
url = 'https://www.tianyancha.com/login'
driver = login('username','password', url)

Login time: 4 second(s)


In [228]:
# Search by company
def search_company(driver, url1):
    driver.get(url1)
    time.sleep(1)
    content = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(content, 'lxml')

    url2 = soup.find('div', class_='header').find('a').attrs['href']
    print('Successfully get the company page url.')

    driver.get(url2)
    return driver

In [250]:
keyword='广州恒大足球'
url_search = 'http://www.tianyancha.com/search?key=%s&checkFrom=searchBox' % keyword
driver = search_company(driver, url_search)

Successfully get the company page url.


In [251]:
def scraper(driver):

    content = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(content, 'lxml')
    
    # Output dictionary
    comp_info={}
    
    comp_info['公司名称'] = soup.find('div', ['header']).find('h1',['name']).string
    print('公司名称：', comp_info['公司名称'])
    
    containers = soup.findAll('div', id=re.compile('^_container_'))
    # print(soup.prettify())
    
    # Containers
    cs = {}
    for container in containers:

        container_id = container.attrs['id'].replace('_container_','')
        #print(container_id)
        cs[container_id]= container
    
    # Get base info
    if 'baseInfo' in cs:
        comp_info['法定代表人'] = cs['baseInfo'].find('div', ['humancompany']).find('div', ['name']).find('a').string
        print('法定代表人：', comp_info['法定代表人'])
        
        comp_info['注册资本'] = cs['baseInfo'].findAll('tr')[4].findAll('td')[1].find('div').get('title')
        print('注册资本：', comp_info['注册资本'])
        
        comp_info['登记地区'] = cs['baseInfo'].findAll('tr')[11].findAll('td')[3].string.split('市')[0]+'市'
        print('登记地区：', comp_info['登记地区'])
        
        #print(cs['baseInfo'].prettify())

    # Get senior management staff
    comp_info['主要人员']=[]
    if 'staff' in cs:
        staff_lst = cs['staff'].findAll(attrs={"event-name": "企业详情-主要人员"})
        for s in staff_lst:
            comp_info['主要人员'].append(s.string)
        print('主要人员：', comp_info['主要人员'])
        #print(cs['staff'].prettify())
    
    #driver.quit()
    
    return comp_info

In [252]:
comp_info = scraper(driver)

公司名称： 广州恒大淘宝足球俱乐部股份有限公司
法定代表人： 高寒
注册资本： 39673.4万人民币
登记地区： 广州市
主要人员： ['高寒', '陈亮', '王帅', '尹雷', '张大钟', '史俊平', '吴泳铭', '姜夫']
